已知关系模式 R=(A,B,C,D,E) 以及函数依赖集合F, 若R在BCNF中，则
1. α-->β，α是β的超集（trivial）
2. α-->β，α是R的超码。


In [69]:
R = ['A','B','C','D','E','F']

F = [[['A'],['B']],[['B'],['C','D']],[['D'],['E']],[['C','E'],['F']]]

In [70]:
def calculate_single_attribute_closure(alpha):
    """
    此函数用于计算单个属性的属性闭包
    ex:
    输入:['A']
    输出:['A','B','C','D','E','F'] 
    """
    if alpha == []:
        return []
    result = alpha.copy()
    while True:
        for alpha_i, beta_i in F:
            #for α_i-->β_i, if α_i ⊂ result 
            if set(alpha_i).issubset(result):
                # result = result ∪ α_i
                result.extend([b for b in beta_i if b not in result])
        #if not changed after a loop, the calculation is finished 
        # 结果不再改变，说明属性闭包计算完成
        if result == alpha:
            break
        #if not finished, update alpha and continue calculations and comparisons 
        # 结果有所改变，继续计算和比较 
        alpha = result 
    return result

In [71]:
def calculate_attribute_closure_set( F ):
    """
    用来计算所有的属性闭包
    ex.
    输入:函数依赖集合F
    输出: 
    ['A']  -->  ['A', 'B', 'C', 'D', 'E', 'F']
    ['B']  -->  ['B', 'C', 'D', 'E', 'F']
    ['D']  -->  ['D', 'E']
    ['C', 'E']  -->  ['C', 'E', 'F']
    """
    #to store all attribute closures 用与存储所有的属性闭包
    attribute_closure_set = []
    for alpha, beta in F:
        result = calculate_single_attribute_closure(alpha)
        #to eliminate duplicate attribute closure 若属性闭包未重复，则添加
        if [alpha,sorted(result)] not in attribute_closure_set:
            attribute_closure_set.append([alpha,sorted(result)])
    return attribute_closure_set 

#calculate all attribute closures
attribute_closure_set = calculate_attribute_closure_set( F ) 

#print all attribute closures
for alpha, alpha_closure in attribute_closure_set:
    print(alpha,' --> ',alpha_closure)


['A']  -->  ['A', 'B', 'C', 'D', 'E', 'F']
['B']  -->  ['B', 'C', 'D', 'E', 'F']
['D']  -->  ['D', 'E']
['C', 'E']  -->  ['C', 'E', 'F']


In [72]:
def calculate_bad_and_good_F():
    """
    Find "bad" F and "good" F for BCNF
    # do it after calculation of attribute_closure_set is finished
    此函数用于计算完所有属性的闭包后, 根据是否为R的超集分成满足或不满足BCNF的函数依赖
    """
    bad_F = []
    good_F = []
    for alpha, alpha_closure in attribute_closure_set:
        for F_alpha, F_beta in F:
            if F_alpha == alpha and not set(R).issubset(alpha_closure):
                bad_F.append([F_alpha, F_beta])
            if F_alpha == alpha and set(R).issubset(alpha_closure):
                good_F.append([F_alpha, F_beta])
    return bad_F, good_F

bad_F, good_F = calculate_bad_and_good_F()
        
# Print bad F
print("Bad F:")
for bad_F_alpha, bad_F_beta in bad_F:
    print(bad_F_alpha,' --> ',bad_F_beta)

# Print good F
print("good F:")
for good_F_alpha, good_F_beta in good_F:
    print(good_F_alpha,' --> ',good_F_beta)


Bad F:
['B']  -->  ['C', 'D']
['D']  -->  ['E']
['C', 'E']  -->  ['F']
good F:
['A']  -->  ['B']


In [73]:
def get_subsets(s):
    """
    此函数用来计算集合的子集
    ex.
    输入:['A','B']
    输出:[[],['A'],['B'],['A','B']]
    """
    if not s:
        return [[]]
    x = s.pop()
    subsets = get_subsets(s)
    return subsets + [subset + [x] for subset in subsets]

def get_proper_subsets(s):
    """
    此函数用来计算集合的真子集
    ex.
    输入:['A','B']
    输出:[[],['A'],['B']]
    """
    temp = s.copy()
    subsets = get_subsets(s)
    subsets.remove(temp)
    return subsets

def determine_candidate_key(alpha):
    """
    此函数用来判断属性集合是否为候选码
    ex.
    输入:['A']
    输出:True
    输入:['B']
    输出:False
    输入:['A','B']
    输出:False(此为超码而不是候选码)
    """
    if(set(R).issubset(calculate_single_attribute_closure(alpha))):

        proper_subsets_of_alpha = get_proper_subsets(alpha)
        sorted_proper_subsets_of_alpha = sorted(proper_subsets_of_alpha, key=len, reverse=True)

        is_candidate_key = True
        for item in sorted_proper_subsets_of_alpha:
            result = calculate_single_attribute_closure(item)
            if set(R).issubset(result):
                is_candidate_key = False
                break
        return is_candidate_key
    return False

def select_candidate_key():
    """
    此函数用来算出候选码
    ex.
    输出:['A']
    """
    candidate_key_set = []
    for alpha, beta in F:
        temp = alpha.copy()
        if(determine_candidate_key(alpha)):
            candidate_key_set.append(temp)
    return candidate_key_set

candidate_key_set = select_candidate_key()

print(candidate_key_set)

[['A']]
